In [271]:
import os
import sys
import numpy as np
import neo
import pandas as pd
import h5py
import ast
import McsPy
import sys, importlib, os
import McsPy.McsData
import McsPy.McsCMOS
from McsPy import ureg, Q_
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz, find_peaks, correlate, gaussian, filtfilt
from scipy import stats
from scipy import signal
from scipy import stats
from scipy import signal
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import McsPy
import McsPy.McsData
from McsPy import ureg, Q_
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import scipy
import time
import glob
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import seaborn as sns
import copy
import pickle
import fnmatch

# Plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns
#networkx
import plotly.graph_objects as go
import networkx as nx
import matplotlib.patches as mpatche


%matplotlib qt

In [272]:
working_directory = '/Users/naila/Documents/DATA/PREPROCESSED/Victoria Witzig/220347CE/output/2022-04-07_cortex_div14_biometra_ID2203047CE_30µMACh_spont_2'

In [273]:
os.chdir(working_directory)

In [274]:
filename=working_directory.split('/')[-1]

In [275]:
glob.glob('*')

['MAIN_RECORDING_Dictionary_2022-04-07_cortex_div14_biometra_ID2203047CE_30µMACh_spont_2.pkl',
 '2022-04-07_cortex_div14_biometra_ID2203047CE_30µMACh_spont_2_full_spikedic.npy',
 'spike_extraction',
 'olde',
 'infodics_2022-04-07_cortex_div14_biometra_ID2203047CE_30µMACh_spont_2.npy',
 '2022-04-07_cortex_div14_biometra_ID2203047CE_30µMACh_spont_2_burst_starts_dictionary.npy']

In [276]:
'MAIN_RECORDING_Dictionary_'+filename+'.pkl'

'MAIN_RECORDING_Dictionary_2022-04-07_cortex_div14_biometra_ID2203047CE_30µMACh_spont_2.pkl'

In [277]:
MAIN_RECORDING_DICTIONARY = pickle.load(open('MAIN_RECORDING_Dictionary_'+filename+'.pkl', "rb"))

In [278]:
MAIN_RECORDING_DICTIONARY.keys()

dict_keys(['spikedic_MAD', 'network_bursts_seconds', 'fr_dic', 'isi_dictionary', 'isi_average_dic', 'isi_std_dic', 'number_rand_notrand_spikes_per_channel', 'rand_notrand_spikes_per_channel', 'Infos_Recording', 'Infos_Analysis', 'Infos_Anatomy', 'Basics', 'Bursts', 'Interburst-Intervals', 'bursting_time_per_channel', 'bursts_per_channel', 'burst_connections'])

In [279]:
spikedic_MAD = MAIN_RECORDING_DICTIONARY['spikedic_MAD']
tick = MAIN_RECORDING_DICTIONARY['Infos_Recording']['tick']
timelengthrecording_s = MAIN_RECORDING_DICTIONARY['Infos_Recording']['timelengthrecording_s']
info_dics_subrecordings = MAIN_RECORDING_DICTIONARY['Infos_Recording']['info_dics_subrecordings']
first_recording_timepoint = info_dics_subrecordings['0_to_120']['first_recording_timepoint']
scale_factor_for_second = MAIN_RECORDING_DICTIONARY['Infos_Recording']['scale_factor_for_second']
relevant_factor = timelengthrecording_s*0.05
fr_dic = MAIN_RECORDING_DICTIONARY['fr_dic']


In [289]:
def remove_artefact_spikes(spikes, recording_length, window=0.01):
    
    removing_times = []
    i = 0
    removed_spikes = {} 
    kicked_out = {}
    while i < timelengthrecording_s:
        print(i)
        activechannels = 0
        for key in spikes:
            for s in spikes[key]:
                if i <= s*tick*scale_factor_for_second < (i + window):
                    activechannels += 1
        if activechannels >= 200:
            removing_times.append(i)
            print(i)
        i += window

    
    print(removing_times)
    for key in spikes:
        new_spikes = []
        kicked_out_list = []

        s_list_save = list(spikes[key])
        s_list = list(spikes[key])
        for s in s_list_save:
            for time in removing_times:
                if time <= s*tick*scale_factor_for_second < (time + window*2):
                    print('time :' + str(time))
                    print('spike :' + str(s*tick*scale_factor_for_second))
                    kicked_out_list.append(s)
                    try:
                        s_list.remove(s)
                        print('removed spike from channel ' + key)
                    except ValueError:
                        print('Value not in List')
                        pass
                #else:
                 #   new_spikes.append(s)
        removed_spikes[key] = s_list
        kicked_out[key] = kicked_out_list

    return removed_spikes, kicked_out

In [290]:
spikedic_MAD, kicked_out = remove_artefact_spikes(spikes=spikedic_MAD, recording_length=timelengthrecording_s)

0
0.01
0.02
0.03
0.04
0.05
0.060000000000000005
0.07
0.08
0.09
0.09999999999999999
0.10999999999999999
0.11999999999999998
0.12999999999999998
0.13999999999999999
0.15
0.16
0.17
0.18000000000000002
0.19000000000000003
0.20000000000000004
0.21000000000000005
0.22000000000000006
0.23000000000000007
0.24000000000000007
0.25000000000000006
0.26000000000000006
0.2700000000000001
0.2800000000000001
0.2900000000000001
0.3000000000000001
0.3100000000000001
0.3200000000000001
0.3300000000000001
0.34000000000000014
0.35000000000000014
0.36000000000000015
0.37000000000000016
0.38000000000000017
0.3900000000000002
0.4000000000000002
0.4100000000000002
0.4200000000000002
0.4300000000000002
0.4400000000000002
0.45000000000000023
0.46000000000000024
0.47000000000000025
0.48000000000000026
0.49000000000000027
0.5000000000000002
0.5100000000000002
0.5200000000000002
0.5300000000000002
0.5400000000000003
0.5500000000000003
0.5600000000000003
0.5700000000000003
0.5800000000000003
0.5900000000000003
0.600

4.509999999999948
4.519999999999948
4.529999999999948
4.539999999999948
4.549999999999947
4.559999999999947
4.569999999999947
4.579999999999947
4.589999999999947
4.599999999999946
4.609999999999946
4.619999999999946
4.629999999999946
4.6399999999999455
4.649999999999945
4.659999999999945
4.669999999999945
4.679999999999945
4.689999999999944
4.699999999999944
4.709999999999944
4.719999999999944
4.729999999999944
4.739999999999943
4.749999999999943
4.759999999999943
4.769999999999943
4.7799999999999425
4.789999999999942
4.799999999999942
4.809999999999942
4.819999999999942
4.8299999999999415
4.839999999999941
4.849999999999941
4.859999999999941
4.869999999999941
4.87999999999994
4.88999999999994
4.89999999999994
4.90999999999994
4.9199999999999395
4.929999999999939
4.939999999999939
4.949999999999939
4.959999999999939
4.9699999999999385
4.979999999999938
4.989999999999938
4.999999999999938
5.009999999999938
5.019999999999937
5.029999999999937
5.039999999999937
5.049999999999937
5.0599999

9.219999999999848
9.229999999999848
9.239999999999847
9.249999999999847
9.259999999999847
9.269999999999847
9.279999999999847
9.289999999999846
9.299999999999846
9.309999999999846
9.319999999999846
9.329999999999846
9.339999999999845
9.349999999999845
9.359999999999845
9.369999999999845
9.379999999999844
9.389999999999844
9.399999999999844
9.409999999999844
9.419999999999844
9.429999999999843
9.439999999999843
9.449999999999843
9.459999999999843
9.469999999999843
9.479999999999842
9.489999999999842
9.499999999999842
9.509999999999842
9.519999999999841
9.529999999999841
9.539999999999841
9.54999999999984
9.55999999999984
9.56999999999984
9.57999999999984
9.58999999999984
9.59999999999984
9.60999999999984
9.61999999999984
9.62999999999984
9.639999999999839
9.649999999999839
9.659999999999838
9.669999999999838
9.679999999999838
9.689999999999838
9.699999999999838
9.709999999999837
9.719999999999837
9.729999999999837
9.739999999999837
9.749999999999837
9.759999999999836
9.769999999999836
9

13.759999999999751
13.76999999999975
13.77999999999975
13.78999999999975
13.79999999999975
13.80999999999975
13.81999999999975
13.82999999999975
13.83999999999975
13.84999999999975
13.859999999999749
13.869999999999749
13.879999999999749
13.889999999999748
13.899999999999748
13.909999999999748
13.919999999999748
13.929999999999747
13.939999999999747
13.949999999999747
13.959999999999747
13.969999999999747
13.979999999999746
13.989999999999746
13.999999999999746
14.009999999999746
14.019999999999746
14.029999999999745
14.039999999999745
14.049999999999745
14.059999999999745
14.069999999999744
14.079999999999744
14.089999999999744
14.099999999999744
14.109999999999744
14.119999999999743
14.129999999999743
14.139999999999743
14.149999999999743
14.159999999999743
14.169999999999742
14.179999999999742
14.189999999999742
14.199999999999742
14.209999999999742
14.219999999999741
14.229999999999741
14.23999999999974
14.24999999999974
14.25999999999974
14.26999999999974
14.27999999999974
14.2899

18.320000000000064
18.330000000000066
18.340000000000067
18.35000000000007
18.36000000000007
18.370000000000072
18.380000000000074
18.390000000000075
18.400000000000077
18.41000000000008
18.42000000000008
18.43000000000008
18.440000000000083
18.450000000000085
18.460000000000086
18.470000000000088
18.48000000000009
18.49000000000009
18.500000000000092
18.510000000000094
18.520000000000095
18.530000000000097
18.5400000000001
18.5500000000001
18.5600000000001
18.570000000000103
18.580000000000105
18.590000000000106
18.600000000000108
18.61000000000011
18.62000000000011
18.630000000000113
18.640000000000114
18.650000000000116
18.660000000000117
18.67000000000012
18.68000000000012
18.690000000000122
18.700000000000124
18.710000000000125
18.720000000000127
18.73000000000013
18.74000000000013
18.75000000000013
18.760000000000133
18.770000000000135
18.780000000000136
18.790000000000138
18.80000000000014
18.81000000000014
18.820000000000142
18.830000000000144
18.840000000000146
18.850000000000

...

286.1999999998847
286.2099999998847
286.2199999998847
286.2299999998847
286.2399999998847
286.24999999988466
286.25999999988466
286.26999999988465
286.27999999988464
286.2899999998846
286.2999999998846
286.3099999998846
286.3199999998846
286.3299999998846
286.3399999998846
286.3499999998846
286.35999999988456
286.36999999988456
286.37999999988455
286.38999999988454
286.3999999998845
286.4099999998845
286.4199999998845
286.4299999998845
286.4399999998845
286.4499999998845
286.4599999998845
286.46999999988446
286.47999999988446
286.48999999988445
286.49999999988444
286.5099999998844
286.5199999998844
286.5299999998844
286.5399999998844
286.5499999998844
286.5599999998844
286.5699999998844
286.57999999988436
286.58999999988436
286.59999999988435
286.60999999988434
286.6199999998843
286.6299999998843
286.6399999998843
286.6499999998843
286.6599999998843
286.6699999998843
286.6799999998843
286.68999999988426
286.69999999988426
286.70999999988425
286.71999999988424
286.7299999998842
286.7399

290.7299999998806
290.7399999998806
290.7499999998806
290.75999999988056
290.76999999988055
290.77999999988054
290.78999999988054
290.7999999998805
290.8099999998805
290.8199999998805
290.8299999998805
290.8399999998805
290.8499999998805
290.8599999998805
290.86999999988046
290.87999999988045
290.88999999988044
290.89999999988044
290.9099999998804
290.9199999998804
290.9299999998804
290.9399999998804
290.9499999998804
290.9599999998804
290.9699999998804
290.97999999988036
290.98999999988035
290.99999999988034
291.00999999988034
291.0199999998803
291.0299999998803
291.0399999998803
291.0499999998803
291.0599999998803
291.0699999998803
291.0799999998803
291.08999999988026
291.09999999988025
291.10999999988024
291.11999999988024
291.1299999998802
291.1399999998802
291.1499999998802
291.1599999998802
291.1699999998802
291.1799999998802
291.1899999998802
291.19999999988016
291.20999999988015
291.21999999988014
291.22999999988014
291.2399999998801
291.2499999998801
291.2599999998801
291.2699

295.1999999998765
295.2099999998765
295.2199999998765
295.2299999998765
295.2399999998765
295.2499999998765
295.25999999987647
295.26999999987646
295.27999999987645
295.28999999987644
295.29999999987643
295.3099999998764
295.3199999998764
295.3299999998764
295.3399999998764
295.3499999998764
295.3599999998764
295.36999999987637
295.37999999987636
295.38999999987635
295.39999999987634
295.40999999987633
295.4199999998763
295.4299999998763
295.4399999998763
295.4499999998763
295.4599999998763
295.4699999998763
295.47999999987627
295.48999999987626
295.49999999987625
295.50999999987624
295.51999999987623
295.5299999998762
295.5399999998762
295.5499999998762
295.5599999998762
295.5699999998762
295.5799999998762
295.58999999987617
295.59999999987616
295.60999999987615
295.61999999987614
295.62999999987613
295.6399999998761
295.6499999998761
295.6599999998761
295.6699999998761
295.6799999998761
295.6899999998761
295.69999999987607
295.70999999987606
295.71999999987605
295.72999999987604
295.

299.7099999998724
299.7199999998724
299.7299999998724
299.7399999998724
299.7499999998724
299.7599999998724
299.76999999987237
299.77999999987236
299.78999999987235
299.79999999987234
299.80999999987233
299.8199999998723
299.8299999998723
299.8399999998723
299.8499999998723
299.8599999998723
299.8699999998723
299.87999999987227
299.88999999987226
299.89999999987225
299.90999999987224
299.91999999987223
299.9299999998722
299.9399999998722
299.9499999998722
299.9599999998722
299.9699999998722
299.9799999998722
299.98999999987217
299.99999999987216
300.00999999987215
300.01999999987214
300.02999999987213
300.0399999998721
300.0499999998721
300.0599999998721
300.0699999998721
300.0799999998721
300.0899999998721
300.09999999987207
300.10999999987206
300.11999999987205
300.12999999987204
300.13999999987203
300.149999999872
300.159999999872
300.169999999872
300.179999999872
300.189999999872
300.199999999872
300.20999999987197
300.21999999987196
300.22999999987195
300.23999999987194
300.249999

304.20999999986833
304.2199999998683
304.2299999998683
304.2399999998683
304.2499999998683
304.2599999998683
304.2699999998683
304.27999999986827
304.28999999986826
304.29999999986825
304.30999999986824
304.31999999986823
304.3299999998682
304.3399999998682
304.3499999998682
304.3599999998682
304.3699999998682
304.3799999998682
304.38999999986817
304.39999999986816
304.40999999986815
304.41999999986814
304.42999999986813
304.4399999998681
304.4499999998681
304.4599999998681
304.4699999998681
304.4799999998681
304.4899999998681
304.49999999986807
[]


In [291]:
spikedic_MAD['G13']

[1014212, 2829194]

In [292]:
kicked_out['G13']

[]

In [293]:
# to use when only relevant channels shall be plotted
'''
active_channels = 0
spikedic_seconds = {}
for key in spikedic_MAD:
    relevant_factor = timelengthrecording_s*0.05
    if len(spikedic_MAD[key])>relevant_factor:
        sec_array = np.asarray(spikedic_MAD[key])*tick*scale_factor_for_second
        spikedic_seconds[key]=sec_array
        active_channels += 1
spikearray_seconds = np.asarray(list(spikedic_seconds.values()))  
'''

# to use when all channels shall be plotted
active_channels = 0
spikedic_seconds = {}
for key in spikedic_MAD:
    sec_array = np.asarray(spikedic_MAD[key])*tick*scale_factor_for_second
    spikedic_seconds[key]=sec_array
    active_channels += 1
spikearray_seconds = np.asarray(list(spikedic_seconds.values()))  


'\nactive_channels = 0\nspikedic_seconds = {}\nfor key in spikedic_MAD:\n    relevant_factor = timelengthrecording_s*0.05\n    if len(spikedic_MAD[key])>relevant_factor:\n        sec_array = np.asarray(spikedic_MAD[key])*tick*scale_factor_for_second\n        spikedic_seconds[key]=sec_array\n        active_channels += 1\nspikearray_seconds = np.asarray(list(spikedic_seconds.values()))  \n'

In [294]:
# for kicked out spikes

active_channels = 0
spikedic_seconds_kicked = {}
for key in kicked_out:
    sec_array = np.asarray(kicked_out[key])*tick*scale_factor_for_second
    spikedic_seconds_kicked[key]=sec_array
    active_channels += 1
spikearray_seconds_kicked = np.asarray(list(spikedic_seconds_kicked.values()))  

In [295]:
spikedic_seconds['G13']

array([101.4212, 282.9194])

In [296]:
spikearray_seconds

array([array([101.4212, 282.9194]), array([ 35.0952,  57.1594, 165.4943]),
       array([], dtype=float64), array([ 62.6127, 182.5041]),
       array([ 46.9488,  52.3411,  63.2546,  64.2372,  65.0019,  65.5723,
               65.9848,  66.4825,  67.0414,  67.044 ,  67.6103,  68.1327,
               68.4931,  68.8673,  69.1956,  69.4282,  69.8795,  70.3318,
               70.7726,  70.7753,  71.0902,  71.8272,  72.1972,  72.5644,
               73.1542,  73.1575,  73.7318,  73.9961,  74.2892,  74.5793,
               74.9067,  74.9175,  75.3399,  75.3431,  75.3487,  75.8225,
               76.1289,  76.1358,  76.229 ,  76.9059,  77.0954,  77.4453,
               77.6124,  77.7717,  78.0388,  78.2907,  79.0272,  80.3641,
               85.9034,  87.0409,  88.6787,  88.8023,  89.4344,  89.8437,
               89.8466,  90.3781,  90.829 ,  91.2758,  91.8019,  92.4832,
               92.9706,  93.7242,  94.1911,  94.5108,  94.9312,  98.5689,
               99.5465, 109.2909, 110.0606, 110.7

In [297]:
len(spikedic_MAD['G13'])

2

In [298]:
relevant_factor

15.225000000000001

In [299]:
"""
# get a 1-D array with every detected spike
scale_factor_for_milisecond = 1e-03
full_spike_list = []
full_spike_list_seconds = []
for key in spikedic_MAD:
    if len(spikedic_MAD[key])>relevant_factor:
        x = list(np.asarray(spikedic_MAD[key])*scale_factor_for_milisecond*tick)
        full_spike_list = full_spike_list + x

        xs = list(np.asarray(spikedic_MAD[key])*scale_factor_for_second*tick)
        full_spike_list_seconds = full_spike_list_seconds + xs
full_spikes = sorted(full_spike_list)
full_spikes_seconds = sorted(full_spike_list_seconds)
"""
scale_factor_for_milisecond = 1e-03
full_spike_list = []
full_spike_list_seconds = []
for key in spikedic_MAD:
    # without the relevant factor --every channel will be plotted
    x = list(np.asarray(spikedic_MAD[key])*scale_factor_for_milisecond*tick)
    full_spike_list = full_spike_list + x

    xs = list(np.asarray(spikedic_MAD[key])*scale_factor_for_second*tick)
    full_spike_list_seconds = full_spike_list_seconds + xs
full_spikes = sorted(full_spike_list)
full_spikes_seconds = sorted(full_spike_list_seconds)

'\n# get a 1-D array with every detected spike\nscale_factor_for_milisecond = 1e-03\nfull_spike_list = []\nfull_spike_list_seconds = []\nfor key in spikedic_MAD:\n    if len(spikedic_MAD[key])>relevant_factor:\n        x = list(np.asarray(spikedic_MAD[key])*scale_factor_for_milisecond*tick)\n        full_spike_list = full_spike_list + x\n\n        xs = list(np.asarray(spikedic_MAD[key])*scale_factor_for_second*tick)\n        full_spike_list_seconds = full_spike_list_seconds + xs\nfull_spikes = sorted(full_spike_list)\nfull_spikes_seconds = sorted(full_spike_list_seconds)\n'

In [300]:
mean_fr_whole_recording = np.around((len(full_spikes_seconds) / timelengthrecording_s), 3)

In [301]:
mean_fr_whole_recording

20.949

In [302]:
len(full_spikes_seconds), len(full_spikes)

(6379, 6379)

In [303]:
 #define bins 
binsize = 0.005 #seconds
#bins= np.arange(0, timelengthrecording_s+binsize, binsize)

# make a histogram 
#full_spikes_binned = np.histogram(full_spikes_seconds, bins)[0]


#trial of population burst plot as inspired by Andrea Corna
bins = int(timelengthrecording_s / binsize)+1

firing_rate_histogram = np.histogram(full_spikes_seconds, bins=bins)
firing_rate = firing_rate_histogram[0]*200 #conversion to hertz
#firing_rate = firing_rate_histogram[0]

In [304]:
bins, len(firing_rate_histogram[0])


(60901, 60901)

In [305]:
def gaussian_smoothing(y, window_size=10, sigma=500):

    filt = signal.gaussian(window_size, sigma)

    return signal.convolve(y, filt, mode='same')


In [306]:
# using firing rate histogram already conversed to hertz

N = int(1/binsize) # für eine Sekunde, das Sliding window, also letztlich number of bins
plot_N = int(0.01/binsize)
# gaussian smmothing fo the firing rate and moving average
fr_gau = gaussian_smoothing(firing_rate)
plot_fr_gau = gaussian_smoothing(firing_rate)

ma_fr_gau = np.convolve(fr_gau, np.ones(N)/N, mode='full')
plotting_ma_fr_gau = np.convolve(fr_gau, np.ones(plot_N)/plot_N, mode='full')


In [307]:
fr_gau[:300]

array([199.9999    , 399.99180016, 399.99420006, 399.99500003,
       399.99420006, 399.99180016, 199.9999    , 199.9991    ,
       199.99750002, 199.99510006, 199.99190016, 199.99190016,
       199.99510006, 399.98940018, 399.99420006, 399.99740002,
       399.999     , 399.999     , 399.99740002, 399.99420006,
       399.98940018, 199.99510006, 199.99190016, 199.99190016,
       199.99510006, 199.99750002, 199.9991    , 199.9999    ,
       199.9999    , 199.9991    , 399.98940018, 399.99020012,
       399.98940018, 199.9991    , 199.9999    , 199.9999    ,
       199.9991    , 199.99750002, 199.99510006, 199.99190016,
       199.99190016, 199.99510006, 199.99750002, 199.9991    ,
       199.9999    , 199.9999    , 199.9991    , 199.99750002,
       199.99510006, 199.99190016,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [308]:
fig = plt.figure(figsize = (10,6))
gs = fig.add_gridspec(3, hspace = 0, height_ratios=[1,1,1])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black', linewidth=0.5)
axs[0].set_ylabel('Firing Rate [Hz]')
axs[1].plot(fr_gau, color = 'red', linewidth=0.5)
axs[2].plot(plotting_ma_fr_gau)

Text(0, 0.5, 'Firing Rate [Hz]')

In [309]:
# we look for the mean of the MA as threshold
# we arrange this mean in an array for plotting
mean_ma_fr_gau = np.mean(ma_fr_gau)
std_ma_fr_gau = np.std(ma_fr_gau)
network_burst_threshold = mean_ma_fr_gau
shape_for_threshold = np.shape(ma_fr_gau)
network_burst_threshold_array = np.full(shape_for_threshold, network_burst_threshold)
    

In [310]:
network_burst_threshold_array

array([208.77233272, 208.77233272, 208.77233272, ..., 208.77233272,
       208.77233272, 208.77233272])

In [311]:
#network_burst_threshold = 1*std_ma_fr_gau

In [312]:
# now we identify the burts from the network and will extract an array with 
# tuples containing the burst start and end times
bursts= []
burst_start = []
burst_seconds_start = []
burst_end = []
burst_seconds_end = []
for index in range(0, len(ma_fr_gau[:-N])):
    if ma_fr_gau[index+N] > network_burst_threshold:
        if ma_fr_gau[index+N-1] <= network_burst_threshold:
            burst_start.append(index)
        if index == 0:
            burst_start.append(0)
            #burst_seconds_start.append((index+N)*0.005)
    else:
        if (ma_fr_gau[index+N-1] > network_burst_threshold) and (len(burst_start)>0):
            if index+N > len(ma_fr_gau):
                ending = len(ma_fr_gau)
            else: 
                ending = index + N

            burst_end.append(ending)
            #burst_seconds_end.append((ending)*0.005)
bursts = list(zip(burst_start, burst_end))
    

In [313]:
for i in burst_start:
    burst_seconds_start.append(firing_rate_histogram[1][i])
for i in burst_end:
    if i >= len(firing_rate_histogram[1]):
        burst_seconds_end.append(firing_rate_histogram[1][-1])
    else:
        burst_seconds_end.append(firing_rate_histogram[1][i])

bursts_seconds = list(zip(burst_seconds_start, burst_seconds_end))
# bursts sind jetzt im 5ms bin   


In [314]:
def zoom_signal(fr, spiketimes, start_in_seconds, stop_in_seconds):
    
    '''
    for zooming in on the plot below
    '''
    
    start = start_in_seconds*200
    stop = stop_in_seconds*200
    
    cutted_fr_array = fr[start:stop+1]
    
    cutted_spiketimes = []
    for channel_spikes in spiketimes:
        s = [y for y in channel_spikes if stop_in_seconds > y >= start_in_seconds]
        cutted_spiketimes.append(s)
        
    return cutted_fr_array, cutted_spiketimes
        
        
        
        
    
    
    

In [315]:
kicked_out

{'G13': [],
 'G12': [],
 'F16': [],
 'F15': [],
 'F14': [],
 'F13': [],
 'F12': [],
 'E16': [],
 'E15': [],
 'E14': [],
 'E13': [],
 'G11': [],
 'D16': [],
 'D15': [],
 'D14': [],
 'H9': [],
 'F11': [],
 'C16': [],
 'C15': [],
 'D13': [],
 'B16': [],
 'B15': [],
 'C14': [],
 'A15': [],
 'A14': [],
 'B14': [],
 'G10': [],
 'E12': [],
 'B13': [],
 'C13': [],
 'F10': [],
 'A13': [],
 'C12': [],
 'D12': [],
 'A12': [],
 'B12': [],
 'D11': [],
 'E11': [],
 'B11': [],
 'C11': [],
 'E10': [],
 'A11': [],
 'C10': [],
 'D10': [],
 'A10': [],
 'B10': [],
 'D9': [],
 'E9': [],
 'B9': [],
 'C9': [],
 'F9': [],
 'A9': [],
 'G8': [],
 'G9': [],
 'A8': [],
 'F8': [],
 'C8': [],
 'B8': [],
 'E8': [],
 'D8': [],
 'B7': [],
 'A7': [],
 'D7': [],
 'C7': [],
 'A6': [],
 'E7': [],
 'C6': [],
 'B6': [],
 'E6': [],
 'D6': [],
 'B5': [],
 'A5': [],
 'D5': [],
 'C5': [],
 'A4': [],
 'F7': [],
 'C4': [],
 'B4': [],
 'F6': [],
 'H8': [],
 'B3': [],
 'A3': [],
 'D4': [],
 'A2': [],
 'B2': [],
 'B1': [],
 'C3': []

In [316]:
fig = plt.figure(figsize = (10,6))
gs = fig.add_gridspec(2, hspace = 0, height_ratios=[2,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black', lw= 0.5)
axs[0].set_ylabel('Firing Rate [Hz]')
axs[0].set_ylim(0, 5500)
axs[1].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5,
                 linelengths = 1, colors = 'black')
axs[1].set_ylabel('Relevant Channels')
fig.suptitle(filename)

#for ax in axs:
#    for i in bursts_seconds:
#        axs[1].axvspan(i[0], i[1], facecolor = '#5B89A6', alpha = 0.3)
#fig.savefig(filename+ '__raster_firingrate_plot_solo.png', dpi=300)
fig.savefig(filename+ '__raster_firingrate_plot_solo.eps', dpi=300)
fig.savefig(filename+ '__raster_firingrate_plot_solo.pdf', dpi=300)
#fig.savefig(filename+ '__raster_firingrate_plot_solo.svg', dpi=300)

Text(0, 0.5, 'Firing Rate [Hz]')

(0.0, 5500.0)

Text(0, 0.5, 'Relevant Channels')

Text(0.5, 0.98, '2022-04-07_cortex_div14_biometra_ID2203047CE_30µMACh_spont_2')

In [233]:
os.getcwd()

'/Users/naila/Documents/DATA/PREPROCESSED/Victoria Witzig/220347CE/output/2022-04-07_cortex_div14_biometra_ID2203047CE_nodrug_spont_2'

In [317]:
# logarhitmic y scale

fig = plt.figure(figsize = (10,6))
gs = fig.add_gridspec(3, hspace = 0, height_ratios=[1,1,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(fr_gau, color= 'black', linewidth=0.5)
axs[0].set_ylabel('Firing Rate [Hz]')
#axs[0].set_yscale('symlog')
axs[0].set_ylim(0, 2000)
axs[1].set_ylim(0, 2000)
axs[1].plot(ma_fr_gau, color = 'black', linewidth=0.5)
axs[1].plot(network_burst_threshold_array, color='red', linewidth = 0.5)
axs[2].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5,
                 linelengths = 2, colors = 'black')
axs[2].set_ylabel('MEA Channels')

#for ax in axs:
 #   for i in bursts_seconds:
  #      axs[2].axvspan(i[0], i[1], facecolor = '#5B89A6', alpha = 0.3)
fig.savefig(filename+ '__raster_firingrate_plot.pdf', dpi=300)
fig.savefig(filename+ '__raster_firingrate_plot.eps', dpi=300)

Text(0, 0.5, 'Firing Rate [Hz]')

(0.0, 2000.0)

(0.0, 2000.0)

Text(0, 0.5, 'MEA Channels')

In [318]:
MAIN_RECORDING_DICTIONARY.keys()

dict_keys(['spikedic_MAD', 'network_bursts_seconds', 'fr_dic', 'isi_dictionary', 'isi_average_dic', 'isi_std_dic', 'number_rand_notrand_spikes_per_channel', 'rand_notrand_spikes_per_channel', 'Infos_Recording', 'Infos_Analysis', 'Infos_Anatomy', 'Basics', 'Bursts', 'Interburst-Intervals', 'bursting_time_per_channel', 'bursts_per_channel', 'burst_connections'])

In [319]:
rand_notrand_spikes_per_channel = MAIN_RECORDING_DICTIONARY['number_rand_notrand_spikes_per_channel']

In [320]:
rand_notrand_spikes_per_channel

{'G13': (0, 2),
 'G12': (2, 1),
 'F16': (0, 0),
 'F15': (1, 1),
 'F14': (92, 195),
 'F13': (0, 1),
 'F12': (0, 0),
 'E16': (0, 0),
 'E15': (0, 0),
 'E14': (0, 0),
 'E13': (2, 1),
 'G11': (1, 0),
 'D16': (0, 3),
 'D15': (1, 0),
 'D14': (2, 0),
 'H9': (4, 2),
 'F11': (0, 0),
 'C16': (1, 1),
 'C15': (1, 0),
 'D13': (0, 2),
 'B16': (0, 0),
 'B15': (1, 0),
 'C14': (22, 142),
 'A15': (1, 0),
 'A14': (1, 5),
 'B14': (1, 1),
 'G10': (0, 1),
 'E12': (3, 0),
 'B13': (2, 3),
 'C13': (4, 14),
 'F10': (0, 0),
 'A13': (7, 9),
 'C12': (9, 15),
 'D12': (5, 16),
 'A12': (1, 6),
 'B12': (2, 1),
 'D11': (13, 21),
 'E11': (1, 0),
 'B11': (20, 188),
 'C11': (2, 140),
 'E10': (2, 0),
 'A11': (6, 12),
 'C10': (1, 5),
 'D10': (19, 102),
 'A10': (0, 1),
 'B10': (1, 1),
 'D9': (1, 1),
 'E9': (0, 17),
 'B9': (0, 1),
 'C9': (3, 3),
 'F9': (0, 0),
 'A9': (1, 0),
 'G8': (0, 2),
 'G9': (0, 2),
 'A8': (2, 1),
 'F8': (2, 0),
 'C8': (22, 96),
 'B8': (2, 0),
 'E8': (1, 0),
 'D8': (1, 12),
 'B7': (3, 5),
 'A7': (4, 2),
 

In [321]:
# identify the longest burst and longest interburst-interval

#longest bursts:

longest_burst = 0
longest_burst_i = 0
for i in bursts_seconds:
    burstinterval = i[1]-i[0]
    if burstinterval > longest_burst:
        longest_burst = burstinterval
        longest_burst_i = i
        
        
# longest_interburst interval:

longest_ibi = 0
longest_ibi_times = (0, 0)

for i in range(1, len(bursts_seconds)-1):
    ibi = bursts_seconds[i+1][0] - bursts_seconds[i][1]
    if ibi > longest_ibi:
        longest_ibi = ibi
        longest_ibi_times = ((bursts_seconds[i][1]), (bursts_seconds[i+1][0])) 

In [322]:
fig = plt.figure(figsize = (20,8))
gs = fig.add_gridspec(2, hspace = 0, height_ratios=[1,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black')
axs[0].set_ylabel('Firing Rate [Hz]')
axs[1].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5, linelengths = 1)
axs[1].set_ylabel('Relevant Channels')

for ax in axs:
    for i in bursts_seconds:
        axs[1].axvspan(i[0], i[1], facecolor = '#118C8C', alpha = 0.3)
        axs[1].axvspan(longest_burst_i[0], longest_burst_i[1], facecolor = '#F2727D', alpha = 0.2, label = 'longest network burst')
        axs[1].axvspan(longest_ibi_times[0], longest_ibi_times[1], facecolor = '#D7D7D9', alpha = 0.1, ls =':')
        
#fig.savefig(filename+'raster_and_FR_plot.png', dpi=300)
        


Text(0, 0.5, 'Firing Rate [Hz]')

Text(0, 0.5, 'Relevant Channels')

In [239]:
# next, we want to isolate these two areas for a spade analysis

In [240]:
def subdivide_spiketrain(spiketrain, sub_start = 0, sub_stop = 10, tick=tick, scale_factor_for_second=1e-06):
    '''
    Excpects: 
        a spiketrain with tick datapoints
        default ticks are 40
        default scale_factor_for_seconds = 1e-06
        provide the start and stop of the desired sub in seconds
    
    Does:
        converts the desired seconds into data ticks
        checks if the spikes of the given spiketrain is in the desired subs
        substracts the starting time -->
        
    Returns:
        a spiketrain dictionary that again starts from zero
    
    '''
    sub_start_tick = sub_start / (tick*scale_factor_for_second)
    sub_stop_tick = sub_stop / (tick*scale_factor_for_second)
    sub_spiketrain = {}
  
    for key in spiketrain: 
        list_per_key = []
        for i in spiketrain[key]:
            if (i>=sub_start_tick ) & (i<sub_stop_tick):
                list_per_key.append(int(i-sub_start_tick))
        sub_spiketrain[key]=list_per_key

    return sub_spiketrain

In [323]:
rand_notrand_spikes_per_channel

{'G13': (0, 2),
 'G12': (2, 1),
 'F16': (0, 0),
 'F15': (1, 1),
 'F14': (92, 195),
 'F13': (0, 1),
 'F12': (0, 0),
 'E16': (0, 0),
 'E15': (0, 0),
 'E14': (0, 0),
 'E13': (2, 1),
 'G11': (1, 0),
 'D16': (0, 3),
 'D15': (1, 0),
 'D14': (2, 0),
 'H9': (4, 2),
 'F11': (0, 0),
 'C16': (1, 1),
 'C15': (1, 0),
 'D13': (0, 2),
 'B16': (0, 0),
 'B15': (1, 0),
 'C14': (22, 142),
 'A15': (1, 0),
 'A14': (1, 5),
 'B14': (1, 1),
 'G10': (0, 1),
 'E12': (3, 0),
 'B13': (2, 3),
 'C13': (4, 14),
 'F10': (0, 0),
 'A13': (7, 9),
 'C12': (9, 15),
 'D12': (5, 16),
 'A12': (1, 6),
 'B12': (2, 1),
 'D11': (13, 21),
 'E11': (1, 0),
 'B11': (20, 188),
 'C11': (2, 140),
 'E10': (2, 0),
 'A11': (6, 12),
 'C10': (1, 5),
 'D10': (19, 102),
 'A10': (0, 1),
 'B10': (1, 1),
 'D9': (1, 1),
 'E9': (0, 17),
 'B9': (0, 1),
 'C9': (3, 3),
 'F9': (0, 0),
 'A9': (1, 0),
 'G8': (0, 2),
 'G9': (0, 2),
 'A8': (2, 1),
 'F8': (2, 0),
 'C8': (22, 96),
 'B8': (2, 0),
 'E8': (1, 0),
 'D8': (1, 12),
 'B7': (3, 5),
 'A7': (4, 2),
 

In [324]:
def get_fr_random_and_non_random_per_channel(random_nrandom_dic, timelength_recording):

    '''
    input:
    random_nrandom_dic = dictionary with
        keys: channellabels, eg G13
        values: tuple with tuple[0] = random detected spikes, 
            and tuple[1]=non random, i.e., network burst detected spike
            
    output:
    2 dic with 
        keys: channellabels
        values:
            fr_random_dic = firing rate in hertz for random spikes (ie, not in network activity)
            fr_nonrandom_dic = firing rate in hertz for nonrandom spikes (ie, in network activity)
    
    '''

    fr_random_dic = {}
    fr_nonrandom_dic = {}
    
    for key in random_nrandom_dic:
        fr_random = random_nrandom_dic[key][0]/timelength_recording
        fr_random_dic[key] = fr_random
        fr_nonrandom = random_nrandom_dic[key][1]/timelength_recording
        fr_nonrandom_dic[key] = fr_nonrandom
        
    return fr_random_dic, fr_nonrandom_dic
        

In [325]:
fr_random_dic, fr_nonrandom_dic = get_fr_random_and_non_random_per_channel(
    random_nrandom_dic=rand_notrand_spikes_per_channel, timelength_recording=timelengthrecording_s)

In [326]:
fr_random_dic

{'G13': 0.0,
 'G12': 0.006568144499178982,
 'F16': 0.0,
 'F15': 0.003284072249589491,
 'F14': 0.3021346469622332,
 'F13': 0.0,
 'F12': 0.0,
 'E16': 0.0,
 'E15': 0.0,
 'E14': 0.0,
 'E13': 0.006568144499178982,
 'G11': 0.003284072249589491,
 'D16': 0.0,
 'D15': 0.003284072249589491,
 'D14': 0.006568144499178982,
 'H9': 0.013136288998357963,
 'F11': 0.0,
 'C16': 0.003284072249589491,
 'C15': 0.003284072249589491,
 'D13': 0.0,
 'B16': 0.0,
 'B15': 0.003284072249589491,
 'C14': 0.0722495894909688,
 'A15': 0.003284072249589491,
 'A14': 0.003284072249589491,
 'B14': 0.003284072249589491,
 'G10': 0.0,
 'E12': 0.009852216748768473,
 'B13': 0.006568144499178982,
 'C13': 0.013136288998357963,
 'F10': 0.0,
 'A13': 0.022988505747126436,
 'C12': 0.029556650246305417,
 'D12': 0.016420361247947456,
 'A12': 0.003284072249589491,
 'B12': 0.006568144499178982,
 'D11': 0.042692939244663386,
 'E11': 0.003284072249589491,
 'B11': 0.06568144499178982,
 'C11': 0.006568144499178982,
 'E10': 0.00656814449917898

In [327]:
# next we plot this on a mea array:

mea_array=np.empty((16,16,))
mea_array[:]=np.nan
columnlist =['A','B','C','D','E','F','G','H','J','K','L','M','N','O','P','R']
df = pd.DataFrame(data=mea_array,columns=columnlist)
df.index+=1

df1 = pd.DataFrame(data=mea_array,columns=columnlist)
df1.index+=1

df2 = pd.DataFrame(data=mea_array,columns=columnlist)
df2.index+=1

df3 = pd.DataFrame(data=mea_array,columns=columnlist)
df3.index+=1

In [328]:
# function to fill the dataframe with the 

def get_dict_values(df, dic):
    
    for i in df.columns:
        for j in range(0, 16):
            keystring = i+str(df.index[j])
            #print(keystring)
            if keystring in dic.keys():
                df.loc[df.index[j],i]=dic[keystring]
            
                
    
    return df

In [329]:
use_df_copy = df.copy()
use_df_copy1 = df1.copy()
use_df_copy2 = df2.copy()
use_df_copy3 = df3.copy()

In [330]:
#the dataframe for burst


df_fr_burst = get_dict_values(use_df_copy, fr_nonrandom_dic)

In [331]:
df_fr_burst

,A,B,C,D,E,F,G,H,J,K,L,M,N,O,P,R
1,NaN,0.003284,0.016420,0.006568,0.000000,0.019704,0.141215,0.213465,0.003284,0.000000,0.000000,0.003284,0.000000,0.000000,0.000000,NaN
2,0.000000,0.009852,0.029557,0.009852,0.003284,0.019704,0.650246,0.022989,0.006568,0.003284,0.003284,0.000000,0.003284,0.003284,0.003284,0.003284
3,0.000000,0.019704,0.000000,0.006568,0.065681,0.022989,0.213465,0.003284,0.019704,0.016420,0.000000,0.000000,0.000000,0.003284,0.006568,0.003284
4,0.000000,0.000000,0.000000,0.003284,0.003284,0.003284,0.042693,0.006568,0.022989,0.019704,0.026273,0.009852,0.006568,0.000000,0.000000,0.000000
5,0.000000,0.114943,0.006568,0.059113,0.006568,0.577997,0.003284,0.000000,0.039409,0.009852,0.108374,0.000000,0.003284,0.000000,0.003284,0.006568
6,0.000000,0.000000,0.006568,0.026273,0.003284,0.009852,0.009852,0.003284,0.009852,0.003284,0.144499,0.006568,0.000000,0.013136,0.052545,0.000000
7,0.006568,0.016420,0.000000,0.006568,0.000000,0.000000,0.000000,0.003284,0.003284,0.003284,0.000000,0.016420,0.003284,0.000000,0.000000,0.003284
8,0.003284,0.000000,0.315271,0.039409,0.000000,0.000000,0.006568,0.016420,0.006568,0.009852,0.003284,0.022989,0.000000,0.003284,0.000000,0.006568
9,0.000000,0.003284,0.009852,0.003284,0.055829,0.000000,0.006568,0.006568,0.003284,0.098522,0.003284,0.003284,0.006568,0.003284,0.003284,0.000000
10,0.003284,0.003284,0.016420,0.334975,0.000000,0.000000,0.003284,0.009852,0.423645,0.755337,0.009852,0.006568,0.006568,0.013136,0.256158,0.019704


In [332]:
# data frame for interburst interval
df_fr_ibi = get_dict_values(use_df_copy1, fr_random_dic)

In [333]:
df_fr_burst

,A,B,C,D,E,F,G,H,J,K,L,M,N,O,P,R
1,NaN,0.003284,0.016420,0.006568,0.000000,0.019704,0.141215,0.213465,0.003284,0.000000,0.000000,0.003284,0.000000,0.000000,0.000000,NaN
2,0.000000,0.009852,0.029557,0.009852,0.003284,0.019704,0.650246,0.022989,0.006568,0.003284,0.003284,0.000000,0.003284,0.003284,0.003284,0.003284
3,0.000000,0.019704,0.000000,0.006568,0.065681,0.022989,0.213465,0.003284,0.019704,0.016420,0.000000,0.000000,0.000000,0.003284,0.006568,0.003284
4,0.000000,0.000000,0.000000,0.003284,0.003284,0.003284,0.042693,0.006568,0.022989,0.019704,0.026273,0.009852,0.006568,0.000000,0.000000,0.000000
5,0.000000,0.114943,0.006568,0.059113,0.006568,0.577997,0.003284,0.000000,0.039409,0.009852,0.108374,0.000000,0.003284,0.000000,0.003284,0.006568
6,0.000000,0.000000,0.006568,0.026273,0.003284,0.009852,0.009852,0.003284,0.009852,0.003284,0.144499,0.006568,0.000000,0.013136,0.052545,0.000000
7,0.006568,0.016420,0.000000,0.006568,0.000000,0.000000,0.000000,0.003284,0.003284,0.003284,0.000000,0.016420,0.003284,0.000000,0.000000,0.003284
8,0.003284,0.000000,0.315271,0.039409,0.000000,0.000000,0.006568,0.016420,0.006568,0.009852,0.003284,0.022989,0.000000,0.003284,0.000000,0.006568
9,0.000000,0.003284,0.009852,0.003284,0.055829,0.000000,0.006568,0.006568,0.003284,0.098522,0.003284,0.003284,0.006568,0.003284,0.003284,0.000000
10,0.003284,0.003284,0.016420,0.334975,0.000000,0.000000,0.003284,0.009852,0.423645,0.755337,0.009852,0.006568,0.006568,0.013136,0.256158,0.019704


In [334]:
f, (ax1, ax2) = plt.subplots(1,2, figsize=(10, 4))

#f.suptitle(filename)
sns.heatmap(df_fr_ibi, annot=False, linewidths=.2, ax=ax1, cmap="rocket_r", vmax=1)
ax1.set_title('firing rate per channel - no bursting [Hz]')


sns.heatmap(df_fr_burst, annot=False, linewidths=.2, ax=ax2, cmap="rocket_r", vmax=1)
ax2.set_title('firing Rate per channel - bursting [Hz]')

f.savefig(filename+'_MEA_Heatmap_burstvsnoburst.eps', dpi = 300)
f.savefig(filename+'_MEA_Heatmap_burstvsnoburst.pdf', dpi = 300)



<AxesSubplot:>

Text(0.5, 1.0, 'firing rate per channel - no bursting [Hz]')

<AxesSubplot:>

Text(0.5, 1.0, 'firing Rate per channel - bursting [Hz]')

In [335]:
f, (ax1, ax2) = plt.subplots(1,2, figsize=(10, 4))

#f.suptitle(filename)
sns.heatmap(df_fr_ibi, annot=False, linewidths=.2, ax=ax1, cmap="rocket_r", vmax=0.5)
ax1.set_title('firing rate per channel - no bursting [Hz]')


sns.heatmap(df_fr_burst, annot=False, linewidths=.2, ax=ax2, cmap="rocket_r", vmax=0.5)
ax2.set_title('firing Rate per channel - bursting [Hz]')

f.savefig(filename+'_MEA_Heatmap_burstvsnoburst_max0p5.eps', dpi = 300)
f.savefig(filename+'_MEA_Heatmap_burstvsnoburst_max0p5.pdf', dpi = 300)


<AxesSubplot:>

Text(0.5, 1.0, 'firing rate per channel - no bursting [Hz]')

<AxesSubplot:>

Text(0.5, 1.0, 'firing Rate per channel - bursting [Hz]')

In [336]:
df_fr_whole = get_dict_values(use_df_copy2, fr_dic)

In [337]:
df_fr_whole

,A,B,C,D,E,F,G,H,J,K,L,M,N,O,P,R
1,NaN,0.009852,0.029557,0.006568,0.009852,0.026273,0.164204,0.331691,0.003284,0.000000,0.000000,0.003284,0.000000,0.000000,0.003284,NaN
2,0.000000,0.049261,0.042693,0.013136,0.016420,0.032841,1.037767,0.049261,0.013136,0.003284,0.003284,0.000000,0.003284,0.009852,0.003284,0.006568
3,0.000000,0.032841,0.003284,0.009852,0.108374,0.045977,0.357964,0.003284,0.022989,0.032841,0.003284,0.000000,0.006568,0.003284,0.006568,0.003284
4,0.000000,0.006568,0.032841,0.009852,0.006568,0.006568,0.098522,0.009852,0.036125,0.032841,0.029557,0.009852,0.009852,0.000000,0.000000,0.003284
5,0.003284,0.190476,0.009852,0.072250,0.013136,0.778325,0.006568,0.006568,0.045977,0.019704,0.167488,0.006568,0.003284,0.013136,0.006568,0.009852
6,0.000000,0.003284,0.009852,0.029557,0.003284,0.013136,0.022989,0.016420,0.019704,0.006568,0.170772,0.019704,0.006568,0.032841,0.072250,0.000000
7,0.019704,0.026273,0.016420,0.009852,0.000000,0.000000,0.003284,0.006568,0.003284,0.009852,0.003284,0.019704,0.009852,0.000000,0.003284,0.003284
8,0.009852,0.006568,0.387521,0.042693,0.003284,0.006568,0.006568,0.019704,0.006568,0.029557,0.003284,0.049261,0.000000,0.006568,0.003284,0.019704
9,0.003284,0.003284,0.019704,0.006568,0.055829,0.000000,0.006568,0.019704,0.016420,0.121511,0.006568,0.006568,0.009852,0.006568,0.009852,0.006568
10,0.003284,0.006568,0.019704,0.397373,0.006568,0.000000,0.003284,0.013136,0.495895,1.021346,0.009852,0.009852,0.006568,0.019704,0.449918,0.026273


In [338]:
'''
sum_mean_frs = 0
number_of_fr = 0
for key in whole_recording_firingrate_dic:
    if whole_recording_firingrate_dic[key] > 0:
        sum_mean_frs += whole_recording_firingrate_dic[key]
        number_of_fr += 1

mean_fr_whole_recording = np.around((sum_mean_frs/number_of_fr), 4)
'''

'\nsum_mean_frs = 0\nnumber_of_fr = 0\nfor key in whole_recording_firingrate_dic:\n    if whole_recording_firingrate_dic[key] > 0:\n        sum_mean_frs += whole_recording_firingrate_dic[key]\n        number_of_fr += 1\n\nmean_fr_whole_recording = np.around((sum_mean_frs/number_of_fr), 4)\n'

In [339]:
mean_fr_whole_recording

20.949

In [340]:
f, ax = plt.subplots(1, 1, figsize = (12,10))

sns.heatmap(df_fr_whole, annot=False, linewidths=.5, ax=ax, cmap="rocket_r", vmax=1)
ax.set_title('firing rate per channel [Hz]')

f.savefig(filename+'whole_MEA_Heatmap.pdf', dpi = 300)
f.savefig(filename+'whole_MEA_Heatmap.eps', dpi = 300)

<AxesSubplot:>

Text(0.5, 1.0, 'firing rate per channel [Hz]')

In [341]:
f, ax = plt.subplots(1, 1, figsize = (12,10))

sns.heatmap(df_fr_whole, annot=False, linewidths=.5, ax=ax, cmap="rocket_r", vmax=0.5)
ax.set_title('firing rate per channel [Hz]')

f.savefig(filename+'whole_MEA_Heatmap_max0p5.pdf', dpi = 300)
f.savefig(filename+'whole_MEA_Heatmap_max0p5.eps', dpi = 300)

<AxesSubplot:>

Text(0.5, 1.0, 'firing rate per channel [Hz]')

In [269]:
filename

'2022-04-07_cortex_div14_biometra_ID2203047CE_nodrug_spont_2'

# Plotting of Layers
Sanity plotting of layerdictionaries

In [125]:
layerdic = {'layer1':["B16","C16","D16","E16","F16","G16","H16","J16","K16","L16","M16","N16","O16","P16"],


            'layer2-3': ["A11","A12","A13","A14","A15","B11","B12","B13","B14","B15","C11","C12","C13","C14","C15",
			"D11","D12","D13","D14","D15","E11","E12","E13","E14","E15","F11","F12","F13","F14","F15",
			"G11","G12","G13","G14","G15","H11","H12","H13","H14","H15","J11","J12","J13","J14","J15",
			"K11","K12","K13","K14","K15","L11","L12","L13","L14","L15","M11","M12","M13","M14","M15",
			"N11","N12","N13","N14","N15","O11","O12","O13","O14","O15","P11","P12","P13","P14","P15",
			"R11","R12","R13","R14","R15"],

           'layer4':    ["A9","A10","B9","B10","C9","C10","D9","D10","E9","E10","F9","F10","G9","G10","H9","H10",
			"J9","J10","K9","K10","L9","L10","M9","M10","N9","N10","O9","O10","P9","P10","R9","R10"],


           'layer5-6':  [	"A2",	"A3",	"A4",	"A5",	"A6",	"A7",	"A8",
"B1",	"B2",	"B3",	"B4",	"B5",	"B6",	"B7",	"B8",
"C1",	"C2",	"C3",	"C4",	"C5",	"C6",	"C7",	"C8",
"D1",	"D2",	"D3",	"D4",	"D5",	"D6",	"D7",	"D8",
"E1",	"E2",	"E3",	"E4",	"E5",	"E6",	"E7",	"E8",
"F1",	"F2",	"F3",	"F4",	"F5",	"F6",	"F7",	"F8",
"G1",	"G2",	"G3",	"G4",	"G5",	"G6",	"G7",	"G8",
"H1",	"H2",	"H3",	"H4",	"H5",	"H6",	"H7",	"H8",
"J1",	"J2",	"J3",	"J4",	"J5",	"J6",	"J7",	"J8",
"K1",	"K2",	"K3",	"K4",	"K5",	"K6",	"K7",	"K8",
"L1",	"L2",	"L3",	"L4",	"L5",	"L6",	"L7",	"L8",
"M1",	"M2",	"M3",	"M4",	"M5",	"M6",	"M7",	"M8",
"N1",	"N2",	"N3",	"N4",	"N5",	"N6",	"N7",	"N8",
"O1",	"O2",	"O3",	"O4",	"O5",	"O6",	"O7",	"O8",
"P1",	"P2",	"P3",	"P4",	"P5",	"P6",	"P7",	"P8",
	"R2",	"R3",	"R4",	"R5",	"R6",	"R7",	"R8"
],

           'whitematter':[]}


In [126]:
def invert_layerdic(layer_dic):
    
    '''
    Expects a dictionary with key = layer, value = list of channellabels
    
    Returns a dictionary with key = channellabels, value = layer
    '''
    layerdic_invert = {}

    for key in layer_dic:
        for i in layer_dic[key]:
            layerdic_invert[i]=key
            
            
    return layerdic_invert

In [127]:
inverted_layerdic = invert_layerdic(layerdic)
inverted_layerdic

{'B16': 'layer1',
 'C16': 'layer1',
 'D16': 'layer1',
 'E16': 'layer1',
 'F16': 'layer1',
 'G16': 'layer1',
 'H16': 'layer1',
 'J16': 'layer1',
 'K16': 'layer1',
 'L16': 'layer1',
 'M16': 'layer1',
 'N16': 'layer1',
 'O16': 'layer1',
 'P16': 'layer1',
 'A11': 'layer2-3',
 'A12': 'layer2-3',
 'A13': 'layer2-3',
 'A14': 'layer2-3',
 'A15': 'layer2-3',
 'B11': 'layer2-3',
 'B12': 'layer2-3',
 'B13': 'layer2-3',
 'B14': 'layer2-3',
 'B15': 'layer2-3',
 'C11': 'layer2-3',
 'C12': 'layer2-3',
 'C13': 'layer2-3',
 'C14': 'layer2-3',
 'C15': 'layer2-3',
 'D11': 'layer2-3',
 'D12': 'layer2-3',
 'D13': 'layer2-3',
 'D14': 'layer2-3',
 'D15': 'layer2-3',
 'E11': 'layer2-3',
 'E12': 'layer2-3',
 'E13': 'layer2-3',
 'E14': 'layer2-3',
 'E15': 'layer2-3',
 'F11': 'layer2-3',
 'F12': 'layer2-3',
 'F13': 'layer2-3',
 'F14': 'layer2-3',
 'F15': 'layer2-3',
 'G11': 'layer2-3',
 'G12': 'layer2-3',
 'G13': 'layer2-3',
 'G14': 'layer2-3',
 'G15': 'layer2-3',
 'H11': 'layer2-3',
 'H12': 'layer2-3',
 'H13': 

In [128]:
def convert_inverted_layerdic_to_numerical(layerdic_inverted):


    layer_numbers={'layer1':2, 
                   'layer2-3':3, 
                    'layer4':4, 
                    'layer5-6':5, 
                    'whitematter':1,
                    'noslice': 0}
    
    numerical_inverted_layerdic = {}
    
    for key in layerdic_inverted:
        layer = layerdic_inverted[key]
        numerical_inverted_layerdic[key]=layer_numbers[layer]
        
    return numerical_inverted_layerdic

In [129]:
num_inverted_layerdic = convert_inverted_layerdic_to_numerical(layerdic_inverted=inverted_layerdic)

In [130]:
df_layers_numbers = get_dict_values(use_df_copy3, num_inverted_layerdic)

In [131]:
df_layers_numbers

,A,B,C,D,E,F,G,H,J,K,L,M,N,O,P,R
1,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN
2,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
4,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
6,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
7,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
8,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
9,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
10,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


In [132]:
f, ax = plt.subplots(1, 1, figsize = (12,10))

sns.heatmap(df_layers_numbers, annot=False, linewidths=.5, ax=ax, 
            cmap="Spectral", vmin=0, vmax=5, cbar=False)
ax.set_title('layers')

f.savefig(filename+'whole_MEA_layers.pdf', dpi = 300)
f.savefig(filename+'whole_MEA_layers.eps', dpi = 300)

<AxesSubplot:>

Text(0.5, 1.0, 'layers')

# Subdivision of a Burst of Interest

In the next part we take a Network Burst and plot it on a small scale to see how the spikes develop. 

Further, we will divide it in 200ms parts to see how the spikes will develop temporarily over the MEA Grid.

In [270]:
# we plot the beginning figure -> burst of interest is in yellow

# we choose the burst we want to plot and get start and end time in seconds

b_of_interest_start = 214.1
b_of_interest_end = 215.1



fig = plt.figure(figsize = (10,4))
gs = fig.add_gridspec(2, hspace = 0, height_ratios=[1,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black')
axs[0].set_ylabel('Firing Rate [Hz]')
axs[1].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5, linelengths = 1)
axs[1].set_ylabel('Relevant Channels')

for ax in axs:
    for i in bursts_seconds:
  #      axs[1].axvspan(i[0], i[1], facecolor = '#118C8C', alpha = 0.3)
        
        axs[1].axvspan(b_of_interest_start, 
                       b_of_interest_end, facecolor = '#F2CD5E', alpha = 0.2, 
                       label = 'longest network burst'
                      )
        
fig.savefig(filename+'raster_and_FR_with_zoomd.eps', dpi=300, bbox_inches = 'tight')
fig.savefig(filename+'raster_and_FR_with_zoomd.pdf', dpi=300, bbox_inches = 'tight')

Text(0, 0.5, 'Firing Rate [Hz]')

Text(0, 0.5, 'Relevant Channels')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [87]:

boi_array = []

for i in spikearray_seconds:
    channel_array_list = []
    for j in i:
        if (j >= b_of_interest_start) and (j <= b_of_interest_end):
            channel_array_list.append(j)
    channel_array = np.asarray(channel_array_list)
    boi_array.append(channel_array)

boi_array = np.asarray(boi_array)

## we can try what happens if we zoom in further

In [88]:
fig, ax = plt.subplots(1, 1, figsize = (10,4))

ax = plt.eventplot(boi_array, color = 'black', linewidths = 0.5, linelengths= 2)
ax = plt.axvspan(b_of_interest_start, 
                       b_of_interest_end, facecolor = '#F2CD5E', alpha = 0.2, 
                       label = 'longest network burst'
                      )

# plot the lines for the subdivisions
#for i in time_windows:
 #   ax = plt.axvline(i[0])
    
    
fig.savefig(filename + 'interesting_burst.eps', dpi=300, bbox_inches = 'tight')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [89]:
# we now want to make heatmaps to picture the activity to observe the spatial propagation

# let us make the heatmap window generic, so the burst is always subdivied into 10 subplots
observed_time = b_of_interest_end-b_of_interest_start

heatmap_window = observed_time/10 # in seconds

# make start and endpoints for each of these time-windows:

time_windows = []

# start of the subdivision = rounded start of the burst of interest
i = np.round(b_of_interest_start, 4)

# for every iteration, add the desired heatmap window
while i < b_of_interest_end:
    start = np.round(i, 4)
    stop = np.round(i+heatmap_window, 4)
    i = i + heatmap_window 
    if stop > b_of_interest_end:
        stop = b_of_interest_end
    time_windows.append((start, stop))


In [90]:
time_windows

[(214.1, 214.2),
 (214.2, 214.3),
 (214.3, 214.4),
 (214.4, 214.5),
 (214.5, 214.6),
 (214.6, 214.7),
 (214.7, 214.8),
 (214.8, 214.9),
 (214.9, 215.0),
 (215.0, 215.1),
 (215.1, 215.1)]

In [91]:
# we again obtain the array of that burst
# in the spikearray_seconds we have every spike used in the whole recording block
# we will now filter them to be in the time region of interest and create burst_of_interest_array

boi_array = []

for i in spikearray_seconds:
    channel_array_list = []
    for j in i:
        if (j >= b_of_interest_start) and (j <= b_of_interest_end):
            channel_array_list.append(j)
    channel_array = np.asarray(channel_array_list)
    boi_array.append(channel_array)

boi_array = np.asarray(boi_array)

In [92]:
fig, ax = plt.subplots(1, 1, figsize = (10,4))

ax = plt.eventplot(boi_array, color = 'black', linewidths = 0.5, linelengths= 2)
ax = plt.axvspan(b_of_interest_start, 
                       b_of_interest_end, facecolor = '#F2CD5E', alpha = 0.2, 
                       label = 'longest network burst'
                      )

# plot the lines for the subdivisions
for i in time_windows:
    ax = plt.axvline(i[0])
    
    
fig.savefig('interesting_burst_zoomed.pdf', dpi=300, bbox_inches = 'tight')
fig.savefig('interesting_burst_zoomed.eps', dpi=300, bbox_inches = 'tight')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [93]:
# we will now create a dictionary with key = starts of the subdivision
# and values = dictionary  of the corresponding spiketrains

subdivided_boi_st = {}
subdivided_boi_fr = {}

for i in time_windows:
    #empty dic for firing rates
    fr_subs = {}
    
    # get the spiketrains for the time subpart
    st = subdivide_spiketrain(spikedic_MAD, i[0], i[1])
    
    # assign the subdivided dictionary to the subdivision dictionary
    subdivided_boi_st[i[0]] = st
    
    # for firing rate, we calculate the fr for each channel
    for key in st:
        fr = len(st[key]) / (i[1]-i[0])
        
        #assign the firing rate to that key
        fr_subs[key] = fr
     
    # the value for the subdivision key is that used subdic, which is emptied in the next loop
    subdivided_boi_fr[i[0]] = fr_subs
        

In [94]:
time_windows = time_windows[:10]

In [95]:
fig = plt.figure(figsize=(24,8))

fig.tight_layout()
fig.suptitle('Burst Propagation from ' + 
             str(np.round(b_of_interest_start, 4))+ ' to ' + 
             str(np.round(b_of_interest_end, 4) ))
gs1 = gridspec.GridSpec(2, 5)

#countries = ["Country " + str(i) for i in range(1, 26)]
axs = []
time_intra_burst = 0

for i in range(1, len(time_windows)+1):
    

    plotframe = copy.deepcopy(df)
    
    # get the key for the fr_dic from the time windows:
    key = time_windows[i-1][0] # substract one because we start the for range at 1
    
    #extract the corresponding fr_dic
    fr_dic = subdivided_boi_fr[key]
    
    # fill the dataframe
    plotframe_filled = get_dict_values(plotframe, fr_dic)
    
    axs.append(fig.add_subplot(gs1[i-1]))
    # plot the heatmap
    #sns.heatmap(plotframe_filled, annot=False, linewidths=.5, ax=axstring, cmap="YlOrRd")

    
        
    axs[-1] = sns.heatmap(plotframe_filled, 
                          annot=False, vmin=0, vmax=50, cmap='rocket_r', 
                          cbar = False, linewidths=0.1, xticklabels=False, yticklabels=False)

    

    plt.setp(axs[-1], 
             title=str(int(time_intra_burst)) + ' ms')
    
    
    time_intra_burst += (heatmap_window*1000)

fig.savefig(filename + 'divided_burst_spatial_zoomed.eps', dpi=300, bbox_inches = 'tight')
fig.savefig(filename + 'divided_burst_spatial_zoomed.pdf', dpi=300, bbox_inches = 'tight')

Text(0.5, 0.98, 'Burst Propagation from 214.1 to 215.1')

[Text(0.5, 1.0, '0 ms')]

[Text(0.5, 1.0, '100 ms')]

[Text(0.5, 1.0, '200 ms')]

[Text(0.5, 1.0, '300 ms')]

...

[Text(0.5, 1.0, '500 ms')]

[Text(0.5, 1.0, '600 ms')]

[Text(0.5, 1.0, '700 ms')]

[Text(0.5, 1.0, '800 ms')]

[Text(0.5, 1.0, '900 ms')]

In [96]:
time_intra_burst = time_windows[0][0]


In [97]:
time_intra_burst

214.1

In [98]:
os.getcwd()

'/Users/naila/Documents/DATA/PREPROCESSED/Jonas Ort/ID046/output/2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_2'